We use Content Planner to generate each section separately.   

There are multiple models (checkpoints) available for stage1 and stage2. We explored all possible combination of 2 models (stage1 and stage2) and the generated results stored in the folder e.g. *nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section1/all_models_generations*     

Similarly to T5, we choose the best combination of models (stage1 and stage2) based on ROUGE score.   


Results of the identified best model save in "/home/ruslan_yermakov/nlg-ra/results/content_planner".

In [1]:
import os

# use metrics from HF
import datasets

In [2]:
# list availbale metrics

from datasets import list_metrics
metrics_list = list_metrics()

metrics_list

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'comet',
 'coval',
 'f1',
 'gleu',
 'glue',
 'indic_glue',
 'meteor',
 'precision',
 'recall',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'squad',
 'squad_v2',
 'super_glue',
 'wer',
 'xnli']

In [3]:
# load rouge metric
metric = datasets.load_metric('rouge')

In [4]:
# example
test1 = ['this is test and I believe it works']
test2 = ['this is test and I hope it works well enough']

metric.add_batch(predictions=test1, references=test2)

# computer the rouge score
res = metric.compute()

{k: round(v.mid.fmeasure * 100, 4) for k, v in res.items()}

{'rouge1': 77.7778, 'rouge2': 62.5, 'rougeL': 77.7778, 'rougeLsum': 77.7778}

In [5]:
def calc_rouge_section(PATH_ORIGINAL, PATH_GENERATED):
    """
    PATH_ORIGINAL - target test generations
    PATH_GENERATED - directory where multiple generated sections are
    """
    
    # read references
    with open(PATH_ORIGINAL) as f:
        gold_references = [line.strip() for line in f]
    
    # keep track of the best generated section and corresponding rouge score
    best_section_filename = ''
    best_section_score = 0
    
    # for each file in PATH_GENERATED
    for filename in os.listdir(PATH_GENERATED):

        # only generations of the stage2
        if filename[:6] == 'stage2':

            # full path to file
            path2file = PATH_GENERATED + filename

            # read candidates
            with open(path2file) as f:
                current_candidates = [line.strip() for line in f]

            # check same length
            assert len(current_candidates) == len(gold_references)

            ### calculate ROUGE scores
            
            # add pairs of predictions/reference to a temporary and memory efficient cache table (HF)
            metric.add_batch(predictions=current_candidates, references=gold_references)
            
            # length of a Metric object will return the number of examples (predictions or predictions/references pair)
            assert len(metric) == len(current_candidates)
            
            # gathers all the cached predictions and references to compute the metric score
            final_score = metric.compute()
            
            # logging
            print(filename, " ~~~ ", {k: round(v.mid.fmeasure * 100, 4) for k, v in final_score.items()})

            final_rouge_scores = {k: round(v.mid.fmeasure * 100, 4) for k, v in final_score.items()}
            
            # get only the rouge2
            rouge2 = final_rouge_scores['rouge2']
            
            # keep track of the best generated section
            if rouge2 > best_section_score:
                best_section_filename = path2file
                best_section_score = rouge2
    
    return (best_section_filename, best_section_score)

In [6]:
## section 1

PATH_ORIGINAL = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section1/test/tgt_test.txt'
PATH_GENERATED = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section1/all_models_generations/'

(best_section1_filename, best_section1_score) = calc_rouge_section(PATH_ORIGINAL, PATH_GENERATED)

print("\n\n ~~~~~~~~~~~ Best generated section1:", best_section1_filename, ' has the ROUGE-2 score: ', best_section1_score)

# save best generated section1 in a separate folder
PATH_KEEP_BEST_SECTION = '/home/ruslan_yermakov/nlg-ra/results/content_planner/test_generations_section1.txt'

with open(best_section1_filename) as reader:
    best_candidates = reader.readlines()

with open(PATH_KEEP_BEST_SECTION, 'w') as file:
    file.write("".join(best_candidates))

stage2_roto_stage1_acc_32.3575_ppl_6049.1894_e12.ptroto_stage2_acc_35.9521_ppl_29.7960_e4.pt.txt  ~~~  {'rouge1': 30.6074, 'rouge2': 11.1491, 'rougeL': 21.7936, 'rougeLsum': 21.8011}
stage2_roto_stage1_acc_35.0000_ppl_32561.3110_e16.ptroto_stage2_acc_55.8835_ppl_12.2270_e12.pt.txt  ~~~  {'rouge1': 43.0825, 'rouge2': 24.5577, 'rougeL': 32.4719, 'rougeLsum': 32.6227}
stage2_roto_stage1_acc_32.1244_ppl_3109.0248_e11.ptroto_stage2_acc_61.1528_ppl_11.1197_e19.pt.txt  ~~~  {'rouge1': 47.0256, 'rouge2': 28.6794, 'rougeL': 35.5915, 'rougeLsum': 35.5811}
stage2_roto_stage1_acc_32.3575_ppl_6049.1894_e12.ptroto_stage2_acc_63.5010_ppl_11.3272_e25.pt.txt  ~~~  {'rouge1': 49.2802, 'rouge2': 30.9936, 'rougeL': 37.7849, 'rougeLsum': 37.7107}
stage2_roto_stage1_acc_35.8290_ppl_31546.7862_e18.ptroto_stage2_acc_35.9521_ppl_29.7960_e4.pt.txt  ~~~  {'rouge1': 30.5806, 'rouge2': 11.2783, 'rougeL': 21.6571, 'rougeLsum': 21.6552}
stage2_roto_stage1_acc_35.0000_ppl_32561.3110_e16.ptroto_stage2_acc_45.1973_ppl_

stage2_roto_stage1_acc_32.1244_ppl_3109.0248_e11.ptroto_stage2_acc_55.8835_ppl_12.2270_e12.pt.txt  ~~~  {'rouge1': 42.2506, 'rouge2': 23.8334, 'rougeL': 31.5313, 'rougeLsum': 31.6298}
stage2_roto_stage1_acc_35.8290_ppl_31546.7862_e18.ptroto_stage2_acc_50.0673_ppl_14.7644_e8.pt.txt  ~~~  {'rouge1': 38.5546, 'rouge2': 18.7025, 'rougeL': 27.1175, 'rougeLsum': 27.1745}
stage2_roto_stage1_acc_35.0000_ppl_32561.3110_e16.ptroto_stage2_acc_59.5425_ppl_11.1062_e16.pt.txt  ~~~  {'rouge1': 45.4416, 'rouge2': 27.6912, 'rougeL': 34.5932, 'rougeLsum': 34.6377}
stage2_roto_stage1_acc_32.1244_ppl_3109.0248_e11.ptroto_stage2_acc_58.4400_ppl_11.5146_e14.pt.txt  ~~~  {'rouge1': 45.2821, 'rouge2': 25.7443, 'rougeL': 33.5686, 'rougeLsum': 33.535}
stage2_roto_stage1_acc_32.1244_ppl_3109.0248_e11.ptroto_stage2_acc_59.5425_ppl_11.1062_e16.pt.txt  ~~~  {'rouge1': 44.5854, 'rouge2': 26.7476, 'rougeL': 33.8801, 'rougeLsum': 33.8184}
stage2_roto_stage1_acc_33.9637_ppl_6288.3673_e14.ptroto_stage2_acc_45.1973_ppl_1

In [7]:
## section 2

PATH_ORIGINAL = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section2/test/tgt_test.txt'
PATH_GENERATED = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section2/all_models_generations/'

(best_section2_filename, best_section2_score) = calc_rouge_section(PATH_ORIGINAL, PATH_GENERATED)

print("\n\n ~~~~~~~~~~~ Best generated section2:", best_section2_filename, ' has the ROUGE-2 score: ', best_section2_score)

# save best generated section1 in a separate folder
PATH_KEEP_BEST_SECTION = '/home/ruslan_yermakov/nlg-ra/results/content_planner/test_generations_section2.txt'

with open(best_section2_filename) as reader:
    best_candidates = reader.readlines()

with open(PATH_KEEP_BEST_SECTION, 'w') as file:
    file.write("".join(best_candidates))

stage2_roto_stage1_acc_27.6637_ppl_80.2429_e6.ptroto_stage2_acc_75.0806_ppl_4.1052_e22.pt.txt  ~~~  {'rouge1': 33.6225, 'rouge2': 17.3793, 'rougeL': 22.5279, 'rougeLsum': 22.5119}
stage2_roto_stage1_acc_18.6016_ppl_57.3508_e4.ptroto_stage2_acc_71.8795_ppl_4.4200_e14.pt.txt  ~~~  {'rouge1': 30.844, 'rouge2': 14.0902, 'rougeL': 19.3138, 'rougeLsum': 19.2739}
stage2_roto_stage1_acc_38.2957_ppl_8571.0033_e20.ptroto_stage2_acc_63.5673_ppl_5.7638_e6.pt.txt  ~~~  {'rouge1': 31.5795, 'rouge2': 16.0245, 'rougeL': 21.144, 'rougeLsum': 21.0925}
stage2_roto_stage1_acc_36.4901_ppl_5194.5648_e15.ptroto_stage2_acc_74.6177_ppl_4.0631_e20.pt.txt  ~~~  {'rouge1': 41.7464, 'rouge2': 22.7151, 'rougeL': 27.0797, 'rougeLsum': 26.9986}
stage2_roto_stage1_acc_33.7186_ppl_1580.8918_e10.ptroto_stage2_acc_68.2577_ppl_5.1150_e10.pt.txt  ~~~  {'rouge1': 37.6771, 'rouge2': 19.5475, 'rougeL': 25.0262, 'rougeLsum': 25.0125}
stage2_roto_stage1_acc_36.4901_ppl_5194.5648_e15.ptroto_stage2_acc_63.5673_ppl_5.7638_e6.pt.tx

stage2_roto_stage1_acc_36.4901_ppl_5194.5648_e15.ptroto_stage2_acc_71.8795_ppl_4.4200_e14.pt.txt  ~~~  {'rouge1': 39.881, 'rouge2': 21.2603, 'rougeL': 25.8384, 'rougeLsum': 25.8322}
stage2_roto_stage1_acc_35.9094_ppl_2029.0636_e12.ptroto_stage2_acc_68.2577_ppl_5.1150_e10.pt.txt  ~~~  {'rouge1': 39.7699, 'rouge2': 20.364, 'rougeL': 25.9388, 'rougeLsum': 25.8793}
stage2_roto_stage1_acc_36.4901_ppl_5194.5648_e15.ptroto_stage2_acc_73.9360_ppl_4.1326_e18.pt.txt  ~~~  {'rouge1': 40.1298, 'rouge2': 21.8234, 'rougeL': 26.3777, 'rougeLsum': 26.3446}
stage2_roto_stage1_acc_18.6016_ppl_57.3508_e4.ptroto_stage2_acc_58.1691_ppl_7.1350_e4.pt.txt  ~~~  {'rouge1': 27.9592, 'rouge2': 11.5521, 'rougeL': 18.9744, 'rougeLsum': 18.977}
stage2_roto_stage1_acc_38.2957_ppl_8571.0033_e20.ptroto_stage2_acc_58.1691_ppl_7.1350_e4.pt.txt  ~~~  {'rouge1': 27.6542, 'rouge2': 13.5943, 'rougeL': 19.2571, 'rougeLsum': 19.1723}
stage2_roto_stage1_acc_33.7186_ppl_1580.8918_e10.ptroto_stage2_acc_70.1979_ppl_4.7599_e12.pt.

In [12]:
## section 3

PATH_ORIGINAL = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section3/test/tgt_test.txt'
PATH_GENERATED = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section3/all_models_generations/'

(best_section3_filename, best_section3_score) = calc_rouge_section(PATH_ORIGINAL, PATH_GENERATED)

print("\n\n ~~~~~~~~~~~ Best generated section3:", best_section3_filename, ' has the ROUGE-2 score: ', best_section3_score)

# save best generated section1 in a separate folder
PATH_KEEP_BEST_SECTION = '/home/ruslan_yermakov/nlg-ra/results/content_planner/test_generations_section3.txt'

with open(best_section3_filename) as reader:
    best_candidates = reader.readlines()

with open(PATH_KEEP_BEST_SECTION, 'w') as file:
    file.write("".join(best_candidates))

stage2_roto_stage1_acc_18.0217_ppl_31625.5055_e13.ptroto_stage2_acc_49.0364_ppl_12.3752_e4.pt.txt  ~~~  {'rouge1': 30.4416, 'rouge2': 12.9743, 'rougeL': 21.4112, 'rougeLsum': 21.3281}
stage2_roto_stage1_acc_12.5715_ppl_435.3259_e7.ptroto_stage2_acc_62.0284_ppl_7.0556_e14.pt.txt  ~~~  {'rouge1': 38.0627, 'rouge2': 20.0778, 'rougeL': 26.363, 'rougeLsum': 26.362}
stage2_roto_stage1_acc_15.1310_ppl_2240.1134_e9.ptroto_stage2_acc_62.0284_ppl_7.0556_e14.pt.txt  ~~~  {'rouge1': 40.7771, 'rouge2': 21.1263, 'rougeL': 28.2864, 'rougeLsum': 28.3011}
stage2_roto_stage1_acc_9.6357_ppl_69.5646_e4.ptroto_stage2_acc_62.0284_ppl_7.0556_e14.pt.txt  ~~~  {'rouge1': 32.8819, 'rouge2': 14.861, 'rougeL': 21.83, 'rougeLsum': 21.8277}
stage2_roto_stage1_acc_12.5715_ppl_435.3259_e7.ptroto_stage2_acc_63.6298_ppl_7.0089_e18.pt.txt  ~~~  {'rouge1': 42.0988, 'rouge2': 23.1903, 'rougeL': 29.3677, 'rougeLsum': 29.3237}
stage2_roto_stage1_acc_19.9789_ppl_198537.7377_e20.ptroto_stage2_acc_60.8357_ppl_7.6152_e12.pt.txt

stage2_roto_stage1_acc_17.8711_ppl_73536.6477_e14.ptroto_stage2_acc_65.2105_ppl_6.7753_e22.pt.txt  ~~~  {'rouge1': 48.1365, 'rouge2': 27.9161, 'rougeL': 34.3217, 'rougeLsum': 34.3376}
stage2_roto_stage1_acc_18.0217_ppl_31625.5055_e13.ptroto_stage2_acc_65.2105_ppl_6.7753_e22.pt.txt  ~~~  {'rouge1': 45.9465, 'rouge2': 26.2753, 'rougeL': 31.9879, 'rougeLsum': 31.9523}
stage2_roto_stage1_acc_19.9789_ppl_198537.7377_e20.ptroto_stage2_acc_65.2105_ppl_6.7753_e22.pt.txt  ~~~  {'rouge1': 48.4368, 'rouge2': 28.5278, 'rougeL': 34.9294, 'rougeLsum': 34.8947}
stage2_roto_stage1_acc_12.5715_ppl_435.3259_e7.ptroto_stage2_acc_54.0132_ppl_9.7249_e6.pt.txt  ~~~  {'rouge1': 34.6531, 'rouge2': 16.8358, 'rougeL': 24.8619, 'rougeLsum': 24.831}
stage2_roto_stage1_acc_15.1310_ppl_2240.1134_e9.ptroto_stage2_acc_54.0132_ppl_9.7249_e6.pt.txt  ~~~  {'rouge1': 35.4655, 'rouge2': 17.2251, 'rougeL': 25.1851, 'rougeLsum': 25.2045}
stage2_roto_stage1_acc_21.0479_ppl_406631.4916_e23.ptroto_stage2_acc_63.6298_ppl_7.0089

In [13]:
## section 4

PATH_ORIGINAL = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section4/test/tgt_test.txt'
PATH_GENERATED = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section4/all_models_generations/'

(best_section4_filename, best_section4_score) = calc_rouge_section(PATH_ORIGINAL, PATH_GENERATED)

print("\n\n ~~~~~~~~~~~ Best generated section4:", best_section4_filename, ' has the ROUGE-2 score: ', best_section4_score)

# save best generated section1 in a separate folder
PATH_KEEP_BEST_SECTION = '/home/ruslan_yermakov/nlg-ra/results/content_planner/test_generations_section4.txt'

with open(best_section4_filename) as reader:
    best_candidates = reader.readlines()

with open(PATH_KEEP_BEST_SECTION, 'w') as file:
    file.write("".join(best_candidates))

stage2_roto_stage1_acc_34.8958_ppl_40.2558_e8.ptroto_stage2_acc_72.4680_ppl_4.0392_e12.pt.txt  ~~~  {'rouge1': 35.7681, 'rouge2': 20.3973, 'rougeL': 24.593, 'rougeLsum': 24.5207}
stage2_roto_stage1_acc_27.2288_ppl_33.5748_e6.ptroto_stage2_acc_78.9941_ppl_3.3024_e24.pt.txt  ~~~  {'rouge1': 32.3466, 'rouge2': 17.1601, 'rougeL': 21.5266, 'rougeLsum': 21.5065}
stage2_roto_stage1_acc_34.8958_ppl_40.2558_e8.ptroto_stage2_acc_66.5604_ppl_5.0249_e8.pt.txt  ~~~  {'rouge1': 33.9737, 'rouge2': 20.1276, 'rougeL': 23.9913, 'rougeLsum': 23.9485}
stage2_roto_stage1_acc_46.8437_ppl_263.3710_e18.ptroto_stage2_acc_74.3791_ppl_3.8232_e14.pt.txt  ~~~  {'rouge1': 42.5472, 'rouge2': 26.8312, 'rougeL': 30.7273, 'rougeLsum': 30.6539}
stage2_roto_stage1_acc_39.8127_ppl_65.8964_e10.ptroto_stage2_acc_78.4621_ppl_3.3857_e22.pt.txt  ~~~  {'rouge1': 38.7795, 'rouge2': 22.059, 'rougeL': 26.4093, 'rougeLsum': 26.3611}
stage2_roto_stage1_acc_48.6515_ppl_340.6895_e24.ptroto_stage2_acc_74.3791_ppl_3.8232_e14.pt.txt  ~~~

stage2_roto_stage1_acc_27.2288_ppl_33.5748_e6.ptroto_stage2_acc_62.0948_ppl_6.1372_e6.pt.txt  ~~~  {'rouge1': 29.5346, 'rouge2': 17.099, 'rougeL': 21.1551, 'rougeLsum': 21.1894}
stage2_roto_stage1_acc_39.8127_ppl_65.8964_e10.ptroto_stage2_acc_74.3791_ppl_3.8232_e14.pt.txt  ~~~  {'rouge1': 37.2348, 'rouge2': 20.9166, 'rougeL': 25.6632, 'rougeLsum': 25.6909}
stage2_roto_stage1_acc_42.4155_ppl_89.4504_e12.ptroto_stage2_acc_66.5604_ppl_5.0249_e8.pt.txt  ~~~  {'rouge1': 36.8874, 'rouge2': 22.3084, 'rougeL': 26.5567, 'rougeLsum': 26.4156}
stage2_roto_stage1_acc_16.2348_ppl_43.5144_e4.ptroto_stage2_acc_72.4680_ppl_4.0392_e12.pt.txt  ~~~  {'rouge1': 26.92, 'rouge2': 13.6302, 'rougeL': 18.3342, 'rougeLsum': 18.2857}
stage2_roto_stage1_acc_39.8127_ppl_65.8964_e10.ptroto_stage2_acc_69.6220_ppl_4.4603_e10.pt.txt  ~~~  {'rouge1': 35.0529, 'rouge2': 19.7188, 'rougeL': 23.8818, 'rougeLsum': 23.8345}
stage2_roto_stage1_acc_46.8437_ppl_263.3710_e18.ptroto_stage2_acc_78.4621_ppl_3.3857_e22.pt.txt  ~~~  

In [14]:
## section 5

PATH_ORIGINAL = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section5/test/tgt_test.txt'
PATH_GENERATED = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section5/all_models_generations/'

(best_section5_filename, best_section5_score) = calc_rouge_section(PATH_ORIGINAL, PATH_GENERATED)

print("\n\n ~~~~~~~~~~~ Best generated section5:", best_section5_filename, ' has the ROUGE-2 score: ', best_section5_score)

# save best generated section1 in a separate folder
PATH_KEEP_BEST_SECTION = '/home/ruslan_yermakov/nlg-ra/results/content_planner/test_generations_section5.txt'

with open(best_section5_filename) as reader:
    best_candidates = reader.readlines()

with open(PATH_KEEP_BEST_SECTION, 'w') as file:
    file.write("".join(best_candidates))

stage2_roto_stage1_acc_47.3934_ppl_7.2179_e5.ptroto_stage2_acc_75.0910_ppl_3.3749_e4.pt.txt  ~~~  {'rouge1': 60.4038, 'rouge2': 49.2453, 'rougeL': 55.3638, 'rougeLsum': 55.3679}
stage2_roto_stage1_acc_48.3412_ppl_14.7150_e8.ptroto_stage2_acc_82.2355_ppl_2.5564_e14.pt.txt  ~~~  {'rouge1': 63.42, 'rouge2': 51.7659, 'rougeL': 57.4064, 'rougeLsum': 57.4892}
stage2_roto_stage1_acc_48.3412_ppl_14.7150_e8.ptroto_stage2_acc_82.8083_ppl_2.7408_e23.pt.txt  ~~~  {'rouge1': 64.2162, 'rouge2': 52.2915, 'rougeL': 57.9257, 'rougeLsum': 57.9977}
stage2_roto_stage1_acc_52.1327_ppl_21.0581_e16.ptroto_stage2_acc_82.5064_ppl_2.5319_e15.pt.txt  ~~~  {'rouge1': 64.5354, 'rouge2': 53.4894, 'rougeL': 58.9635, 'rougeLsum': 58.8813}
stage2_roto_stage1_acc_50.5924_ppl_14.3700_e12.ptroto_stage2_acc_82.6457_ppl_2.6186_e18.pt.txt  ~~~  {'rouge1': 63.98, 'rouge2': 53.115, 'rougeL': 58.5559, 'rougeLsum': 58.4584}
stage2_roto_stage1_acc_50.8294_ppl_16.4651_e13.ptroto_stage2_acc_82.8083_ppl_2.7408_e23.pt.txt  ~~~  {'ro

stage2_roto_stage1_acc_49.2891_ppl_31.8245_e15.ptroto_stage2_acc_75.0910_ppl_3.3749_e4.pt.txt  ~~~  {'rouge1': 59.6117, 'rouge2': 48.2338, 'rougeL': 54.7061, 'rougeLsum': 54.7265}
stage2_roto_stage1_acc_48.3412_ppl_14.7150_e8.ptroto_stage2_acc_82.6457_ppl_2.6186_e18.pt.txt  ~~~  {'rouge1': 64.3317, 'rouge2': 52.6268, 'rougeL': 58.0007, 'rougeLsum': 58.0432}
stage2_roto_stage1_acc_52.8436_ppl_32.1332_e18.ptroto_stage2_acc_80.4861_ppl_2.6043_e10.pt.txt  ~~~  {'rouge1': 63.2292, 'rouge2': 51.6192, 'rougeL': 57.552, 'rougeLsum': 57.5237}
stage2_roto_stage1_acc_51.7773_ppl_35.6314_e19.ptroto_stage2_acc_81.3685_ppl_2.5736_e12.pt.txt  ~~~  {'rouge1': 63.6309, 'rouge2': 52.0928, 'rougeL': 57.8532, 'rougeLsum': 57.724}
stage2_roto_stage1_acc_52.1327_ppl_21.0581_e16.ptroto_stage2_acc_82.7541_ppl_2.6229_e17.pt.txt  ~~~  {'rouge1': 64.294, 'rouge2': 53.1974, 'rougeL': 58.5332, 'rougeLsum': 58.5424}
stage2_roto_stage1_acc_47.3934_ppl_7.2179_e5.ptroto_stage2_acc_82.2355_ppl_2.5564_e14.pt.txt  ~~~  {

stage2_roto_stage1_acc_50.8294_ppl_16.4651_e13.ptroto_stage2_acc_79.6966_ppl_2.7064_e8.pt.txt  ~~~  {'rouge1': 62.1185, 'rouge2': 50.9256, 'rougeL': 56.9774, 'rougeLsum': 56.9288}
stage2_roto_stage1_acc_49.2891_ppl_31.8245_e15.ptroto_stage2_acc_82.7541_ppl_2.6229_e17.pt.txt  ~~~  {'rouge1': 64.1739, 'rouge2': 53.2827, 'rougeL': 58.6786, 'rougeLsum': 58.6878}
stage2_roto_stage1_acc_52.1327_ppl_21.0581_e16.ptroto_stage2_acc_82.6457_ppl_2.6186_e18.pt.txt  ~~~  {'rouge1': 63.9009, 'rouge2': 52.7443, 'rougeL': 58.0894, 'rougeLsum': 58.0775}
stage2_roto_stage1_acc_54.0284_ppl_51.3860_e20.ptroto_stage2_acc_82.5064_ppl_2.5319_e15.pt.txt  ~~~  {'rouge1': 65.4487, 'rouge2': 54.4243, 'rougeL': 59.7895, 'rougeLsum': 59.7442}
stage2_roto_stage1_acc_50.8294_ppl_16.4651_e13.ptroto_stage2_acc_77.4905_ppl_2.9508_e6.pt.txt  ~~~  {'rouge1': 58.7802, 'rouge2': 47.1378, 'rougeL': 53.5919, 'rougeLsum': 53.5698}
stage2_roto_stage1_acc_52.8436_ppl_32.1332_e18.ptroto_stage2_acc_77.4905_ppl_2.9508_e6.pt.txt  ~~

In [15]:
## section 6

PATH_ORIGINAL = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section6/test/tgt_test.txt'
PATH_GENERATED = '/home/ruslan_yermakov/nlg-ra/reproducibility/totto_data2text_plan_py/paper_code_section6/all_models_generations/'

(best_section6_filename, best_section6_score) = calc_rouge_section(PATH_ORIGINAL, PATH_GENERATED)

print("\n\n ~~~~~~~~~~~ Best generated section6:", best_section6_filename, ' has the ROUGE-2 score: ', best_section6_score)

# save best generated section1 in a separate folder
PATH_KEEP_BEST_SECTION = '/home/ruslan_yermakov/nlg-ra/results/content_planner/test_generations_section6.txt'

with open(best_section6_filename) as reader:
    best_candidates = reader.readlines()

with open(PATH_KEEP_BEST_SECTION, 'w') as file:
    file.write("".join(best_candidates))

stage2_roto_stage1_acc_21.2229_ppl_73.1081_e6.ptroto_stage2_acc_66.8564_ppl_7.2482_e20.pt.txt  ~~~  {'rouge1': 41.3761, 'rouge2': 25.1489, 'rougeL': 33.2561, 'rougeLsum': 33.3159}
stage2_roto_stage1_acc_24.0631_ppl_217.8215_e8.ptroto_stage2_acc_66.8564_ppl_7.2482_e20.pt.txt  ~~~  {'rouge1': 43.7401, 'rouge2': 26.8954, 'rougeL': 35.6951, 'rougeLsum': 35.6249}
stage2_roto_stage1_acc_24.0631_ppl_217.8215_e8.ptroto_stage2_acc_65.6575_ppl_7.1781_e16.pt.txt  ~~~  {'rouge1': 41.3586, 'rouge2': 25.8886, 'rougeL': 33.8615, 'rougeLsum': 33.8271}
stage2_roto_stage1_acc_24.0631_ppl_217.8215_e8.ptroto_stage2_acc_57.8079_ppl_7.8213_e6.pt.txt  ~~~  {'rouge1': 31.8056, 'rouge2': 18.5544, 'rougeL': 25.5545, 'rougeLsum': 25.5383}
stage2_roto_stage1_acc_26.2130_ppl_2519.1899_e12.ptroto_stage2_acc_66.8564_ppl_7.2482_e20.pt.txt  ~~~  {'rouge1': 47.2738, 'rouge2': 30.4752, 'rougeL': 39.1184, 'rougeLsum': 39.2201}
stage2_roto_stage1_acc_29.1519_ppl_82949.3649_e20.ptroto_stage2_acc_60.0880_ppl_7.3363_e8.pt.tx

stage2_roto_stage1_acc_27.9882_ppl_5882.3857_e14.ptroto_stage2_acc_64.0868_ppl_6.8235_e12.pt.txt  ~~~  {'rouge1': 42.1708, 'rouge2': 25.9741, 'rougeL': 34.7074, 'rougeLsum': 34.6491}
stage2_roto_stage1_acc_29.0730_ppl_80313.8291_e18.ptroto_stage2_acc_65.6575_ppl_7.1781_e16.pt.txt  ~~~  {'rouge1': 46.8958, 'rouge2': 30.4229, 'rougeL': 39.034, 'rougeLsum': 39.065}
stage2_roto_stage1_acc_24.0631_ppl_217.8215_e8.ptroto_stage2_acc_62.2316_ppl_7.1621_e10.pt.txt  ~~~  {'rouge1': 39.5941, 'rouge2': 24.0005, 'rougeL': 32.1252, 'rougeLsum': 32.0161}
stage2_roto_stage1_acc_27.9882_ppl_5882.3857_e14.ptroto_stage2_acc_52.0487_ppl_10.3840_e4.pt.txt  ~~~  {'rouge1': 33.5639, 'rouge2': 19.2731, 'rougeL': 27.4824, 'rougeLsum': 27.4969}
stage2_roto_stage1_acc_21.2229_ppl_73.1081_e6.ptroto_stage2_acc_57.8079_ppl_7.8213_e6.pt.txt  ~~~  {'rouge1': 30.0802, 'rouge2': 17.613, 'rougeL': 24.5999, 'rougeLsum': 24.6772}
stage2_roto_stage1_acc_26.2130_ppl_2519.1899_e12.ptroto_stage2_acc_57.8079_ppl_7.8213_e6.pt.t